### Purpose of the Code:
#### The goal of the code is to optimize the location of drugstores (Drogerien) in the city of Salzburg so that every residential building is assigned to one nearby store (within walking distance of 1500 meters), while minimizing the number of newly opened stores. The model is implemented using Mixed Integer Programming (MIP) and solved using Python (with PuLP).

#### Constraints:

1. Each building must be assigned to exactly one drugstore.

2. A building can only be assigned to an open (selected) drugstore.

3. Maximum walking distance is 1500 meters between building and store.

4. Limit on the number of drugstores per district:

- Usually max 4 per district, but up to 5 in "Leopoldskron/Moos" due to low density and long distances.

5. For districts with existing drugstores, the model only optimizes for unserved buildings, using only new locations and adjusting the limit accordingly.

6. If too many drugstores are selected, the model automatically keeps the ones that serve the most buildings within the 1500 m range.


In [ ]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Load a matrix of distances from each point to every other point in the city, prepared by QGIS, with the correct delimiter
data = pd.read_csv(r"C:/Users/Yoanna/Downloads/notnntalfreisaal_OD.csv", delimiter=';')

# Check the columns
print(data.columns)

# Constraint for maximum distance (cost)
max_total_cost = 1500

# Transform data into a convenient format
origin_ids = data['origin_id'].unique()        # All origin_id (buildings)
destination_ids = data['destination_id'].unique()  # All destination_id (drugstore locations)

# Optimization model
model = LpProblem("Minimize_Number_Of_Drugstores", LpMinimize)

# Variables:
# Binary variable for each destination_id (whether a drugstore is opened)
x = {destination: LpVariable(f"x_{destination}", cat="Binary") for destination in destination_ids}

# Binary variable for each link (origin_id -> destination_id)
y = {(origin_id, destination): LpVariable(f"y_{origin_id}_{destination}", cat="Binary")
     for origin_id in origin_ids for destination in destination_ids}

# Objective function: minimize the number of open drugstores
model += lpSum(x[destination] for destination in destination_ids)

# Constraints:
# 1. Each origin_id must be connected to exactly one drugstore
for origin_id in origin_ids:
    model += lpSum(y[(origin_id, destination)] for destination in destination_ids) == 1, f"Cover_Origin_{origin_id}"

# 2. Connection (origin_id -> destination_id) is only allowed if the destination drugstore is open
for origin_id in origin_ids:
    for destination in destination_ids:
        model += y[(origin_id, destination)] <= x[destination], f"Link_Origin_{origin_id}_Destination_{destination}"

# 3. Disallow connections where distance (cost) exceeds the maximum allowed
for _, row in data.iterrows():
    origin_id = row['origin_id']
    destination_id = row['destination_id']
    if row['total_cost'] > max_total_cost:
        model += y[(origin_id, destination_id)] == 0, f"Cost_Limit_{origin_id}_{destination_id}"

# Solve the model
model.solve()

# Get results
opened_drugstores = [destination for destination in destination_ids if x[destination].value() == 1]

print(f"Opened drugstores are located at: {opened_drugstores}")

# For each open drugstore, print the buildings assigned to it
for destination in opened_drugstores:
    assigned_buildings = [origin_id for origin_id in origin_ids if y[(origin_id, destination)].value() == 1]
    print(f"Drugstore at location {destination} serves the following buildings: {assigned_buildings}")


Index(['origin_id', 'destination_id', 'entry_cost', 'network_cost',
       'exit_cost', 'total_cost'],
      dtype='object')


### Checking whether each building is connected to a built drugstore

In [ ]:
if uncovered_origins:
    print(f"The following buildings do not have a valid connection to any drugstore: {uncovered_origins}")
    raise ValueError("Not all buildings can be covered.")
else:
    print("All buildings have a valid connection to at least one drugstore.")


In [ ]:
# Check if all origin_id are assigned to at least one of the opened drugstores
unassigned_origins = []

for origin_id in origin_ids:
    # Check if this origin_id is assigned to at least one of the opened drugstores
    is_assigned = False
    for destination in opened_drugstores:
        if y[(origin_id, destination)].value() == 1:
            is_assigned = True
            break
    if not is_assigned:
        unassigned_origins.append(origin_id)

# If there are unassigned origin_id, print them
if unassigned_origins:
    print(f"Unassigned buildings (origin_id): {unassigned_origins}")
else:
    print("All origin_id are assigned to at least one of the opened drugstores.")


### Limiting the Number of Open Drugstores to the Top 4 Based on Coverage
#### Since Salzburg has districts whose buildings are very sparsely distributed, it was necessary to place a restriction in which no more than 4 drugstores could be built in a district. 

In [ ]:
# Check if the number of opened drugstores is greater than 4
if len(opened_drugstores) > 4:
    # Create a dictionary with the number of buildings served by each drugstore
    drugstore_assignments = {destination: sum(y[(origin_id, destination)].value() for origin_id in origin_ids)
                             for destination in opened_drugstores}

    # Sort the drugstores by number of buildings served (from most to least)
    sorted_drugstores = sorted(drugstore_assignments.items(), key=lambda item: item[1], reverse=True)

    # Select the top 4 drugstores
    top_drugstores = {item[0] for item in sorted_drugstores[:4]}

    # Remove drugstores that are not in the top 4
    for destination in opened_drugstores:
        if destination not in top_drugstores:
            x[destination].varValue = 0
            for origin_id in origin_ids:
                y[(origin_id, destination)].varValue = 0

    # Update the list of opened drugstores
    opened_drugstores = list(top_drugstores)

# Print the result
print(f"Final opened drugstores (limited to 4): {opened_drugstores}")

# For each opened drugstore, list the origin_id values assigned to it
for destination in opened_drugstores:
    assigned_buildings = [origin_id for origin_id in origin_ids if y[(origin_id, destination)].value() == 1]
    print(f"Drugstore at location {destination} serves the following buildings: {assigned_buildings}")


### When there are existing drugstores in the district
#### Since there are of course districts in Salzburg where there are already existing drugstores, a separate code has been created for these districts that allocates residential buildings (that are less than 1500 meters away) to the already built drugstores, and then finds a solution for the remaining buildings

In [3]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Load the data
data = pd.read_csv(r"C:\Users\Yoanna\Downloads\itzhagen_OD.csv", delimiter=';')

# Maximum allowed distance (total cost)
max_total_cost = 1500

# Pre-existing drugstores (provide your destination_id values)
existing_drugstores = ["way/956550043"]  # Replace with actual IDs of existing drugstores

# Determine whether each origin is covered by an existing drugstore within max distance
data['Covered'] = data.apply(
    lambda row: row['destination_id'] in existing_drugstores and row['total_cost'] <= max_total_cost,
    axis=1
)

# Filter origin_ids that are covered
covered_origin_ids = data[data['Covered']]['origin_id'].unique()

# Identify origin_ids that are not covered
all_origin_ids = data['origin_id'].unique()
uncovered_origin_ids = [id_ for id_ in all_origin_ids if id_ not in covered_origin_ids]

# Print summary of coverage
print(f"Total number of buildings (origin_id): {len(all_origin_ids)}")
print(f"Number of buildings that are covered: {len(covered_origin_ids)}")
print(f"Number of buildings that are not covered: {len(uncovered_origin_ids)}")


Total number of buildings (origin_id): 520
Number of buildings that are covered: 404
Number of buildings that are not covered: 116


In [4]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Load the data
data = pd.read_csv(r"C:\Users\Yoanna\Downloads\itzhagen_OD.csv", delimiter=';')

# Maximum allowed distance
max_total_cost = 1500

# Pre-existing drugstores (list of destination_id)
existing_drugstores = ["way/956550043"]

# Separate covered and uncovered buildings
covered_origin_ids = set()
for drugstore in existing_drugstores:
    covered_origin_ids.update(
        data[(data['destination_id'] == drugstore) & (data['total_cost'] <= max_total_cost)]['origin_id']
    )

# List of origin_ids that need to be served
uncovered_data = data[~data['origin_id'].isin(covered_origin_ids)]

# Unique origin_ids and destination_ids for uncovered locations
uncovered_origin_ids = uncovered_data['origin_id'].unique()
destination_ids = uncovered_data['destination_id'].unique()

# Optimization model
model = LpProblem("Minimize_Total_Cost", LpMinimize)

# Variables
x = {destination: LpVariable(f"x_{destination}", cat="Binary") for destination in destination_ids}  # New drugstores
y = {(origin, destination): LpVariable(f"y_{origin}_{destination}", cat="Binary")
     for origin in uncovered_origin_ids for destination in destination_ids}  # Assignments

# Objective: minimize the number of newly opened drugstores
model += lpSum(x[destination] for destination in destination_ids)

# Constraint 1: Every uncovered building must be connected to exactly one drugstore
for origin in uncovered_origin_ids:
    model += lpSum(y[(origin, destination)] for destination in destination_ids) == 1

# Constraint 2: A building can only be connected to an open drugstore
for origin in uncovered_origin_ids:
    for destination in destination_ids:
        model += y[(origin, destination)] <= x[destination]

# Constraint 3: Limit the maximum distance between a building and a drugstore
for _, row in uncovered_data.iterrows():
    origin_id, destination_id, total_cost = row['origin_id'], row['destination_id'], row['total_cost']
    if total_cost > max_total_cost:
        model += y[(origin_id, destination_id)] == 0

# Solve the model
model.solve()

# Results
newly_opened_drugstores = [destination for destination in destination_ids if x[destination].value() == 1]

print(f"New drugstores will be opened at the following locations: {newly_opened_drugstores}")

# For each new drugstore, display the buildings it serves
for destination in newly_opened_drugstores:
    assigned_buildings = [origin for origin in uncovered_origin_ids if y[(origin, destination)].value() == 1]
    print(f"Drugstore at location {destination} serves the following buildings: {assigned_buildings}")



New drugstores will be opened at the following locations: ['way/1017795419']
Drugstore at location way/1017795419 serves the following buildings: ['way/58414623', 'way/58414639', 'way/58414641', 'way/58414643', 'way/85220894', 'way/97237534', 'way/112784382', 'way/112784383', 'way/112784385', 'way/112954253', 'way/115274731', 'way/115274776', 'way/115274861', 'way/115441052', 'way/115441063', 'way/115441064', 'way/115441073', 'way/115441075', 'way/115441079', 'way/115441083', 'way/115441094', 'way/115441100', 'way/115441102', 'way/115441105', 'way/115441109', 'way/115441113', 'way/115441125', 'way/115441129', 'way/115441130', 'way/117232596', 'way/117232598', 'way/117232599', 'way/117232600', 'way/117232602', 'way/117232603', 'way/117232605', 'way/117232606', 'way/117232608', 'way/117232609', 'way/117232610', 'way/117232612', 'way/117232613', 'way/117232616', 'way/117232621', 'way/117232622', 'way/117232623', 'way/117232624', 'way/117232626', 'way/117232627', 'way/117232629', 'way/1172